# Final Project
## Students: Nafis Banirazi & Jan Carbonell
### Lab Objective:
The Objectives of this project are the following

In [1]:
#intial set of imports
import nltk
from nltk import pos_tag
from nltk.corpus import wordnet, stopwords
from nltk.metrics import jaccard_distance
from nltk.stem import WordNetLemmatizer
from nltk.wsd import lesk
import string

#variable initialization and instantiation
d = {}
j = [0]
tests = []
corpus = []
tests = []
golden_std = []

## 1. Read all pairs of sentences of the train and test set
We proceed to open and read the input file

In [2]:
def text_to_sentences(filename):
    sentence_pair_array = []
    for line in open(filename, encoding="UTF8").readlines():
        sentence_pair_array.append([s.strip() for s in line.split("\t")])
    return sentence_pair_array


train_input = text_to_sentences('../00_data/trial/STS.input_fixed.txt')
#train_input = text_to_sentences('../00_data/train/STS.input.MSRpar.txt')
test_input = text_to_sentences('../00_data/test-gold/STS.input.MSRpar.txt')

train_classes = open('../00_data/trial/STS_fixed.gs.txt', encoding="utf-8-sig").readlines()
#train_classes = open('../00_data/train/STS.gs.MSRpar.txt', encoding="utf-8-sig").readlines()
test_classes = open('../00_data/test-gold/STS.gs.MSRpar.txt', encoding="UTF8").readlines()

## 2. Noisy entities removal functions
### Stopwords, URL's, Punctuations

In [3]:
# Removing the punctuation and lowering the case of a string
def remove_punctuation(line):
    return line.translate(str.maketrans('', '', string.punctuation))

## 3. Word normalization
### Tokenization, Lemmatization, Stemming

In [4]:
# extract the words from the sentence minus stopwords
def words_from_sent(sent):
    # tokenized
    tokenized = nltk.word_tokenize(sent)
    # remove stopwords and return
    return [word for word in tokenized if word not in nltk.corpus.stopwords.words('english')]

# convert words to tokens
def tokens_from_words(words):
    return pos_tag(words)

# Function to get wordnet pos code
def wordnet_pos_code(tag):
    if tag.startswith('NN'):
        return wordnet.NOUN
    elif tag.startswith('VB'):
        return wordnet.VERB
    elif tag.startswith('JJ'):
        return wordnet.ADJ
    elif tag.startswith('RB'):
        return wordnet.ADV
    else:
        return None

# Tokens to lemmas using wordnet lemmatizer    
def tokens_to_lemmas(tokens):
    return list(map(token_to_lemmas, tokens))

def token_to_lemmas(token):    
    pos = wordnet_pos_code(token[1])
    if pos:
        return WordNetLemmatizer().lemmatize(token[0], pos=pos)
    return token[0]


## 4. Word Standarization
### Regular Expression 


## Evaluation

In [5]:
%matplotlib inline

In [6]:
# evaluation imports
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, mean_squared_error, r2_score

# from sklearn import preprocessing
# from sklearn import utils
# from sklearn.svm import SVR

#True/False Positive: https://uberpython.wordpress.com/2012/01/01/precision-recall-sensitivity-and-specificity/

def count(g,s):
    TP = TN = FP = FN = 0
    for i in range(0,len(g)):
        if (g[i]==s[i] and s[i]==1): TP+=1
        if (g[i]==s[i] and s[i]==0): TN+=1
        if (g[i]!=s[i] and s[i]==1): FP+=1
        if (g[i]!=s[i] and s[i]==0): FN+=1
    return [TP,TN,FP,FN]

def MSRP_eval(gs, sys):
    [TP,TN,FP,FN] = count(gs,sys)
    acc = (TP+TN)/float(TP+TN+FP+FN) # ACCURACY
    reject = TN/float(TN+FP) # precision on negative SPECIFICITY
    accept = TP/float(TP+FN) # precision on positive SENSITIVITY
    print("acc=",acc," reject=",reject," accept=",accept)

## 5. Main program


In [7]:
'''
    First Paraphrase detector approach, jaccard distance of lemmas    
'''
def lemma_jaccard(sent_0, sent_1):
    #print('.', end='')
    sent_0, sent_1 = remove_punctuation(sent_0.lower()), remove_punctuation(sent_1.lower())
    print(sent_0 + '\n' + sent_1 + '\n')
    words_0, words_1 = words_from_sent(sent_0), words_from_sent(sent_1)
    tokens_0, tokens_1 = tokens_from_words(words_0), tokens_from_words(words_1)
    lemmas_0, lemmas_1 = tokens_to_lemmas(tokens_0), tokens_to_lemmas(tokens_1)
    
    #jaccard_similarity
    print (5 *float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1))), '\n')
    return 5*float(1 - jaccard_distance(set(lemmas_0), set(lemmas_1)))


In [8]:
print('Training')
training_data_X = [lemma_jaccard(data[0], data[1]) for data in train_input]
training_scores_Y = [float(line.strip()) for line in train_classes]
print('Finished Training!\n')

print('Testing')
testing_data_X = [lemma_jaccard(data[0], data[1])for data in test_input]
testing_scores_Y = [float(line.strip()) for line in test_classes]
print('Finished Testing!\n')

print('Results')
#maybe label encoding https://www.kaggle.com/pratsiuk/valueerror-unknown-label-type-continuous
print(training_data_X)
print('########################################################################\n')
print(np.array(training_data_X).reshape(-1,1))
print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n')
print(training_scores_Y)

#creating linear regression object
regression = linear_model.LinearRegression()

# Train the model using the training sets
regression.fit(np.array(training_data_X).reshape(-1,1), training_scores_Y)

# Make predictions using the testing set
prediction = regression.predict(np.array(testing_data_X).reshape(-1,1))
# MSRP_eval(prediction, 1000*np.array(testing_scores_Y))
print('Results are ready!\n')

Training
the bird is bathing in the sink
birdie is washing itself in the water basin

0.0 

in may 2010 the troops attempted to invade kabul
the us army invaded kabul on may 7th last year 2010

1.25 

john said he is considered a witness but not a suspect
he is not a suspect anymore john said

2.5 

they flew out of the nest in groups
they flew into the nest together

2.5 

the woman is playing the violin
the young lady enjoys listening to the guitar

0.0 

john went horse back riding at dawn with a whole group of friends
sunrise at dawn is a magnificent view to take in if you wake up early enough for it

0.3125 

Finished Training!

Testing
the problem likely will mean corrective changes before the shuttle fleet starts flying again
he said the problem needs to be corrected before the space shuttle fleet is cleared to fly again

1.4285714285714284 

the technologylaced nasdaq composite index ixic inched down 1 point or 011 percent to 1650
the broad standard  poors 500 index spx inched 

3.5714285714285716 

earlier they had defied a police order and cried allahu akbar god is greatest as bashir walked to his seat in the tightly guarded courtroom
earlier bashirs supporters had defied a police order and cried allahu akbar god is greatest as he walked to his seat

2.7777777777777777 

metropolitan ambulance spokesman james howe said five people were taken to hospital and three were treated at the scene after yesterdays incident
spokesman james howe said five children aged between 4 and 17 were taken to hospital with neck and chest injuries while three others were treated at the scene

1.875 

chiron already has nearly 20 percent acceptances from powderjects shareholders
chiron has acceptances from holders of nearly 20 percent of powderject shares

2.272727272727273 

but it is clear that the joint intelligence committee was not involved
this report was cleared by the joint intelligence committee

3.333333333333334 

about 22 of twentysomethings are obese which is roughly 

2.0 

north american markets finished mixed in directionless trading monday as earnings season begins to slow and economic indicators move into the spotlight
north american markets grabbed early gains monday morning as earnings season begins to slow and economic indicators take the spotlight

2.619047619047619 

as well as the dolphin scheme the chaos has allowed foreign companies to engage in damaging logging and fishing operations without proper monitoring or export controls
internal chaos has allowed foreign companies to set up damaging commercial logging and fishing operations without proper monitoring or export controls

2.8571428571428568 

the value will total about 124 million including convertible securities according to corel
including convertible securities the total estimated value of the deal is 124 million according to corel

4.090909090909091 

the standard  poors 500 stock index ended the quarter up 120 points a gain of 14 percent the best performance for that broad mar

3.076923076923077 

the 30year bond us30ytrr jumped 2032 taking its yield to 414 percent from 418 percent
the 30year bond us30ytrr dipped 1432 for a yield of 426 percent from 423 percent

1.7857142857142856 

but he is the only candidate who has won national labor endorsements so far picking up his 11th union on tuesday when the united steelworkers of america endorsed him
but he is the only candidate who has won national labor endorsements so far picking up his 11th on tuesday

3.076923076923077 

oracle shares fell 27 cents or 2 percent to 1309
oracle shares also rose on the news up 15 cents to 1351 on the nasdaq

1.0714285714285716 

north american futures pointed to a subpar start to trading on tuesday with investors ready to get their first taste of quarterly earnings
north american stock markets got off to a slow start tuesday with investors ready to get their first taste of quarterly earnings

3.055555555555556 

another million barrels bought by spanish refiner cepsa sa was to b

2.0588235294117645 

further out the curve the benchmark 10year note us10ytrr shed 2532 in price taking its yield to 326 percent from 317 percent
early wednesday the benchmark 10year us10ytrr had lost 1632 in price driving its yield up to 333 percent from 326 percent late tuesday

1.666666666666667 

tech stocks were hurt by a sour forecast from sun microsystems which was viewed as a bad omen for the upcoming quarterly earnings season
a sour forecast from sun microsystems inc sunwo put more pressure on wall street before the quarterly earnings season

1.75 

trading in loral was halted yesterday the shares closed on monday at 301
the new york stock exchange suspended trading yesterday in loral which closed at 301 friday

1.7857142857142856 

shares of coke were up 6 cents at 4442 in afternoon trading wednesday on the new york stock exchange
shares of coke were down 26 cents to close at 4410 on the new york stock exchange

2.3333333333333335 

ernst  young has denied any wrongdoing and 

...

1.7999999999999998 

artists are worried the plan would harm those who need help most  performers who have a difficult time lining up shows
the artists say the plan will harm french culture and punish those who need help most  performers who have a hard time lining up work

1.5000000000000002 

long lines formed outside gas stations and people rushed to get money from cash machines sunday as israelis prepared to weather a strike that threatened to paralyze the country
long lines formed sunday outside gas stations and people rushed to get money from cash machines as israelis braced for the strikes effects

3.409090909090909 

the techheavy nasdaq composite index fell 399 or 02 percent to 168272 following a twoday win of 5593
the technologylaced nasdaq composite index ixic eased 852 points or 051 percent to 167021

0.9999999999999998 

martin 58 will be freed today after serving two thirds of his fiveyear sentence for the manslaughter of 16yearold fred barras
martin served two thirds of 

1.7857142857142856 

president bush raised a recordbreaking 495 million for his reelection campaign over the last three months with contributions from 262000 americans the presidents campaign chairman said tuesday
president bush has raised 839 million since beginning his reelection campaign in may and has 70 million of that left to spend his campaign said tuesday

1.666666666666667 

authorities in ohio indiana and michigan have searched for the bodies
so far authorities also have searched areas in pennsylvania ohio indiana and michigan

2.5 

he said european governments have blocked all new biocrops because of unfounded unscientific fears
they have blocked all new biocrops because of unfounded unscientific fears bush said

3.5 

one question was whether france which infuriated washington by leading the charge against un authorization for the war would vote yes or abstain
france which infuriated washington by leading the charge against un approval for the war also sought changes

1.84

1.4285714285714284 

evacuation went smoothly although passengers werent told what was going on hunt said
passengers were evacuated smoothly although they were not told what was going on he said

2.272727272727273 

the discovery that the map bug is present in the vast majority of crohns sufferers means it is almost certainly causing the intestinal inflammation it said in a statement
the researchers say that the fact the map bug is present in the vast majority of crohns sufferers means it is almost certainly causing the intestinal inflammation

3.888888888888889 

between 1993 and 2002 142 allegations of sexual assault were reported
from 1993 to 2002 there were 142 reported sexual assaults at the academy

3.75 

smith said simply oh my god in the seconds afterward according to weinshall
in the seconds after the crash she added captain smith said simply oh my god

2.5 

the life expectancy for boys born in 2001 was 744 years up two years since 1990
according to the report average life e

1.1538461538461537 

the us senate judiciary committee overcame a significant hurdle yesterday in the battle to create a trust fund to pay victims of asbestos exposure
the senate judiciary committee on tuesday overcame a significant hurdle in the battle to create a trust fund to pay victims of asbestos exposure but the most difficult obstacles remain

3.5 

but she didnt say whether shell certify the twoyear 1174 billion budget for 200405
she said she didnt know yet whether she would certify the budget

1.923076923076923 

it would be difficult to overestimate the potential dangers of the remote procedure call rpc vulnerability
the flaw involves the remote procedure call rpc protocol which deals with intercomputer communications

1.25 

although obesity can increase the risk of health problems skeptics argue so do smoking and high cholesterol
although obesity can increase the risk of a host of health problems skeptics argue so do smoking and high cholesterol which are not considered di

3.6363636363636367 

the man who entered the post office in lakeside shortly before 3 pm gave up to deputies about 630 pm
the man had entered the post office on woodside avenue at maine avenue shortly before 3 pm

2.5 

fewer than 10 fbi offices have conducted investigations involving visits to mosques the justice department said
in addition the justice department said that the fbi has conducted fewer than 10 investigations involving visits to mosques

4.230769230769231 

outside the court sriyanto who faces up to 25 years jail denied he had done anything wrong
outside the court sriyanto denied to the age that he had done anything wrong

2.916666666666666 

university of michigan president mary sue coleman said in a statement on the universitys web site our fundamental values havent changed
our fundamental values havent changed mary sue coleman president of the university said in a statement in ann arbor

3.2352941176470584 

two astronomers surveying the region around jupiter have det

In [9]:
pearsonr(training_scores_Y, testing_scores_Y)[0]

ValueError: operands could not be broadcast together with shapes (6,) (750,) 

In [ ]:
#confusion_matrix(training_scores_Y, testing_scores_Y)
pearsonr(training_scores_Y, testing_scores_Y)[0]


# The coefficients
print('Coefficients: \n', regression.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(testing_scores_Y, training_scores_Y))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(testing_scores_Y, training_scores_Y))

# Plot outputs
plt.scatter(testing_data_X , testing_scores_Y ,  color='black')
plt.plot(testing_data_X, training_scores_Y, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()